In [35]:
from scipy.integrate import odeint
from scipy.optimize import minimize

import matplotlib.pyplot as plot
import pandas as pd
import numpy as np
plot.style.use('seaborn')

experiment = 'test_calibration'

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [ ]:
def electric_heater(T, t, parameters, u):
    # Parameters
    # U_loss = heat losses to air
    # A = area of heat Transfer
    # T_amb = ambient temperature
    # gain = gain

    U_loss = parameters[0]  # W / m^2 s
    gain = parameters[1]  #
    A = 20  # m^2
    T_amb = 25  # Celsius
    R_mass = 50  # grams
    Cpr = 1  # J / g K

    Q_loss = A * U_loss * (T - T_amb)
    dTrdt = (gain * u - Q_loss) / (R_mass * Cpr)

    return dTrdt


def simulate(parameters):
    initial_conditions = data.loc[0, "T_heater"]
    simulation_results = pd.DataFrame(columns=["Time", "u", "T_heater"])
    simulation_results.loc[0, "Time"] = data.loc[0, "Time"]
    simulation_results.loc[0, "T_heater"] = initial_conditions
    simulation_results.loc[0, "u"] = data.loc[0, "PWM"]

    N = len(data)
    time = data.loc[0:, "Time"]
    u = data.loc[0:, "PWM"]

    for i in range(1, N):
        time_interval = [time[i - 1], time[i]]
        solution = odeint(
            electric_heater, initial_conditions, time_interval, args=(parameters, u[i - 0])
        )

        simulation_results.loc[i, "Time"] = time[i]
        simulation_results.loc[i, "T_heater"] = solution[-1][0]
        simulation_results.loc[i, "u"] = u[i]
        initial_conditions = solution[-1][0]

    return simulation_results


def squared_error(simulation_results):
    y = data.loc[:, "T_heater"].to_numpy()
    yhat = simulation_results.loc[:, "T_heater"].to_numpy()
    penalty = np.diag(0.8 * np.ones(len(data)))

    return np.dot(np.dot((y - yhat).transpose(), penalty), (y - yhat))


def model_deviation(simulation_results):
    yhat = simulation_results.loc[:, "T_heater"].to_numpy()
    yhat_old = old_results.loc[:, "T_heater"].to_numpy()
    penalty = np.diag(0.1 * np.ones(len(data)))

    return np.dot(np.dot((yhat - yhat_old).transpose(), penalty), (yhat - yhat_old))


def parameter_movement(parameters):
    p = np.array(parameters)
    p_old = parameters_df.iloc[-1, :].to_numpy()
    penalty = np.diag(0.1 * np.ones(len(p_old)))
    delta_p = (p - p_old)

    return np.dot(np.dot(delta_p.transpose(), penalty), delta_p)


def objective(parameters):
    simulation_results = simulate(parameters)
    objective = squared_error(simulation_results) + model_deviation(simulation_results) + parameter_movement(parameters)

    return objective / len(data)

In [ ]:
parameters_df = pd.DataFrame(columns=["U_loss", "Gain"])
parameters_df.loc[0, "U_loss"] = 1e-5   # Heat losses to air
parameters_df.loc[0, "Gain"] = 10  # Heat gain from electric current
iter = 0

In [ ]:
%time
iter = iter + 1

# Load Data
data = pd.read_csv(experiment + '/data.csv')
data.loc[:, "Time"] = data.loc[:, "Time"] * 3600
data = data.reset_index().drop(columns=['index'])

# Simulate model with parameters
old_parameters = parameters_df.loc[iter - 1, :].to_list()
old_results = simulate(old_parameters)
new_parameters = minimize(objective, old_parameters, method='Nelder-Mead', tol=100)
parameters_df.loc[iter, :] = new_parameters.x

In [ ]:
plot.plot(data.loc[:, "Time"], data.loc[:, "T_heater"], old_results.loc[:, "Time"], old_results.loc[:, "T_heater"], 'r')
plot.show()